## Лабораторные занятия по численным методам

|                       |                             |
|:----------------------|:---------------------------:|
| Подготовили           | Смирнова Л. и Глуховский М. |
| Преподаватель         | Шабунина Зоя Александровна  |
| Лабораторная работа   |      № 1 Задание 12**       |
| Язык программирования |           Python            |

---

### Задание 12 (**).
**Назначение.** Численные исследования сходимости глобальных интерполяционных процессов для непрерывных на отрезке функций.

**Метод.** Для визуального исследования сходимости интерполяционных процессов разработать процедуру, которая выводит на экран компьютера два графика на заданном отрезке – график заданной функции
`f(x)` и график глобального интерполяционного многочлена Фейера,
построенного для этой функции на сетке чебышевских узлов. Входными параметрами этой процедуры являются концы отрезка интерполирования, количество чебышевских узлов на этом отрезке, непрерывная функция `f(x)`.

**Замечание.**
Вычислительные эксперименты должны быть проведены в том числе
для функций `1/(1 + 25x²)`, `|x|`.

**Указание.** См. `[1]` настоящего пособия.

---


In [2]:
# Подключение бибылиотек
from pydantic import BaseModel, PositiveInt, ValidationError, root_validator
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
from sympy.abc import a, b, x, i, n
from sympy import cos, pi, Abs

from pathlib import Path

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
%config IPCompleter.greedy=True

In [3]:
class InpData(BaseModel):
    a: float | int
    b: float | int
    number_nodes: PositiveInt
        
    @root_validator
    def check_segment_a_b(cls, values):
        a, b = values.get('a'), values.get('b')
        assert a <= b, f'invalid segment [{a}, {b}]'
        return values

try:
    inp_data = InpData.parse_file(Path('input.json'))
except ValidationError as e:
    print("ОШИБКА ВХОДНЫХ ДАННЫХ")
    print(e.json())
    raise e

### Функция

In [ ]:
# f_x = 1/ (1 + 25 * x**2)
# f_x = Abs(x)
# f_x = x ** 7 - x**6 + 5*x**5 - 3*x**4 + x
f_x = x ** 2
f_x

### Чебышевские узлы

In [ ]:
def get_node(a: sp.core.symbol.Symbol | float | int = sp.symbols('a'), 
             b: sp.core.symbol.Symbol | float | int = sp.symbols('b'), 
             i: sp.core.symbol.Symbol | float | int = sp.symbols('i')
            ) -> sp.core.add.Add:
    x_i: sp.core.add.Add = (a + b) / 2 + ((b - a) / 2) * cos(((2*i + 1)/(2*i + 2))*pi)
    return x_i

def get_nodes(a: sp.core.symbol.Symbol | float | int = sp.symbols('a'), 
              b: sp.core.symbol.Symbol | float | int = sp.symbols('b'), 
              n: int = 0) -> list:
    x_i: sp.core.add.Add = get_node(a, b)
    return [x_i.subs(i, i_) for i_ in range(n)]


get_node()

nodes: list = get_nodes(a=inp_data.a, b=inp_data.b, n=inp_data.number_nodes)

for node in get_nodes(a=a, b=b, n=3):
    node
    
print(f"Чебышевские узлы для a = {inp_data.a}, b = {inp_data.b}, кол. узлов = {inp_data.number_nodes}")
for node in nodes:
    node

### Глобальный интерполяционный многочлен Фейера, построенный для этой функции на сетке чебышевских узлов

In [ ]:
def get_H(nodes: list | tuple,
          f=None, 
          f_arg: sp.core.symbol.Symbol = sp.symbols('x'), 
          x: sp.core.symbol.Symbol | float | int = sp.symbols('x')
         ) -> sp.core.add.Add:
    
    H: sp.core.add.Add = 0
        
    for x_i in nodes:
        a_i = 0
        for x_k in nodes: 
            if x_k != x_i:
                a_i += 1 / (x_i - x_k)
        a_i *= -2
        
        
        b_i = 1 - a_i * x_i
        
        numerator: sp.core.mul.Mul = sp.prod([(x - xi_)**2 for xi_ in nodes if xi_ != x_i]) * (a_i * x + b_i)
        denominator: sp.core.mul.Mul = sp.prod([(x_i - xi_)**2 for xi_ in nodes if xi_ != x_i])

        if not f:
            f_x_i = sp.Function('f')(x_i)
        else:
            f_x_i = f.subs(f_arg, x_i)
            
        H += f_x_i * numerator / denominator

    return H

nodes_symbols: tuple = sp.var(f'x0:{inp_data.number_nodes}')
get_H(nodes_symbols)

# print("Для чебышевских узлов и функции f(x): ")
H_x: sp.core.add.Add = get_H(nodes, f_x)
# H_x

### График заданной функции `f(x)` и график глобального интерполяционного многочлена Фейера, построенного для этой функции на сетке чебышевских узлов


In [ ]:
plot_1 = sp.plotting.plot(H_x, (x, inp_data.a, inp_data.b), line_color='red', show=False, title='Графики функций f(x) и H(x)') 
plot_2 = sp.plotting.plot(f_x, (x, inp_data.a, inp_data.b), line_color='blue', show=False)
plot_1.append(plot_2[0])
plot_1.show()

### Графики на отдельных изображениях

In [ ]:
sp.plotting.plot(f_x, (x, inp_data.a, inp_data.b), title='График функции f(x)')
sp.plotting.plot(H_x, (x, inp_data.a, inp_data.b), title='График функции H(x)', line_color='red')

### Вся логика вместе, с выводом только графиков

In [ ]:
def get_H(nodes: list | tuple,
          f=None, 
          f_arg: sp.core.symbol.Symbol = sp.symbols('x'), 
          x: sp.core.symbol.Symbol | float | int = sp.symbols('x')
         ) -> sp.core.add.Add:
    
    H: sp.core.add.Add = 0
        
    for x_i in nodes:
        a_i = 0
        for x_k in nodes: 
            if x_k != x_i:
                a_i += 1 / (x_i - x_k)
        a_i *= -2
        
        
        b_i = 1 - a_i * x_i
        
        numerator: sp.core.mul.Mul = sp.prod([(x - xi_)**2 for xi_ in nodes if xi_ != x_i]) * (a_i * x + b_i)
        denominator: sp.core.mul.Mul = sp.prod([(x_i - xi_)**2 for xi_ in nodes if xi_ != x_i])

        if not f:
            f_x_i = sp.Function('f')(x_i)
        else:
            f_x_i = f.subs(f_arg, x_i)
            
        H += f_x_i * numerator / denominator

    return H

try:
    inp_data = InpData.parse_file(Path('input.json'))
except ValidationError as e:
    print("ОШИБКА ВХОДНЫХ ДАННЫХ")
    print(e.json())
    raise e


f_x = x

nodes: list = get_nodes(a=inp_data.a, b=inp_data.b, n=inp_data.number_nodes)

H_x: sp.core.add.Add = get_H(nodes, f_x)

plot_1 = sp.plotting.plot(H_x, (x, inp_data.a, inp_data.b), line_color='red', show=False, title='Графики функций f(x) и H(x)') 
plot_2 = sp.plotting.plot(f_x, (x, inp_data.a, inp_data.b), line_color='blue', show=False)
plot_1.append(plot_2[0])
plot_1.show()

In [ ]:
sp.plotting.plot(f_x, (x, inp_data.a, inp_data.b), title='График функции f(x)')
sp.plotting.plot(H_x, (x, inp_data.a, inp_data.b), title='График функции H(x)', line_color='red')

In [ ]:
### По той же формуле, в другом виде

In [ ]:
def get_H(nodes: list | tuple,
          f=None, 
          f_arg: sp.core.symbol.Symbol = sp.symbols('x'), 
          x: sp.core.symbol.Symbol | float | int = sp.symbols('x')
         ) -> sp.core.add.Add:
    
    H: sp.core.add.Add = 0
        
    for x_i in nodes:
        
        numerator: sp.core.mul.Mul = sp.prod([(x - xi_)**2 for xi_ in nodes if xi_ != x_i]) * (x - x_i)
        denominator: sp.core.mul.Mul = sp.prod([(x_i - xi_)**2 for xi_ in nodes if xi_ != x_i])
                            
        a_i = 0
        for x_k in nodes: 
            if x_k != x_i:
                a_i += 1 / (x_i - x_k)
        a_i *= -2

        b_i = 1 - a_i * x_i
    
    
        if not f:
            f_x_i = sp.Function('f')(x_i)
        else:
            f_x_i = f.subs(f_arg, x_i)
    
    
        H += (a_i * x + b_i) * f_x_i* numerator / denominator
    
    
    return H

nodes_symbols: tuple = sp.var(f'x0:{inp_data.number_nodes}')
get_H(nodes_symbols)

# print("Для чебышевских узлов и функции f(x): ")
H_x: sp.core.add.Add = get_H(nodes, f_x)
# H_x